In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from scipy.special import expit
from scipy.optimize import fmin, minimize
import glob as glob
import matplotlib
matplotlib.style.use('ggplot')

# .py
%load_ext autoreload
%autoreload 2
import weather_all_alphas, weather_all_costs, prediction_humidity, model_02, validate, dengue_weather

running model_02
getting cost from nweek ahead
getting bounded alphas with params for avgrh...
done getting alphas ...
getting cost from nweek ahead
prediction for humidity
dengue weather


In [2]:
def seperate_train_test(all_data,train_data):
    plt.axvspan(0, len(train_data), color='blue', alpha=0.1)
    plt.text((len(train_data)/2)-25,700,'Train Period',size=16)
    plt.axvline(len(train_data)+1,linewidth=0.5, color='green')
    plt.text(len(train_data)+20,700,'Test Period',size=16)
    plt.axvspan(len(train_data)+1, len(all_data), color='green', alpha=0.1)

def plot_all_predictions(LAG, real, train, province, nweeks_to_predict, all_predictions):
    
    plt.figure(figsize=(13,6))
    plt.plot(range(len(real)),real.cases,label='real')
    graph_title = "%s's Dengue Prediction with Weather Data"%province
    plt.title(graph_title)
    for i in range(len(all_predictions)):
        cur_week_guess = nweeks_to_predict[i]
        start_predict_week = LAG+(cur_week_guess-1)
        predicted_cases = all_predictions[i]
        x_weeks = range(start_predict_week,len(predicted_cases)+start_predict_week)
        line_label = "%d week guess"%cur_week_guess
        plt.plot(x_weeks,predicted_cases,label=line_label,alpha=0.8)
    
    seperate_train_test(real,train)
    plt.axvline(LAG)
    plt.text((LAG*2)+10,300,'Prediction Start',horizontalalignment='center',verticalalignment='center')
    plt.xlabel("# week starting 2003")
    plt.ylabel("Dengue cases")
    plt.legend()
#     plt.savefig("16-lags-16-weeks-9-temp-wks-chiangmai-00.png")
    
def show_predictions(params, real, train, province, ws):
    LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, AVGRH_WEEKS = params[0], params[1], params[2], params[3]
    
    predictions_to_plot = prediction_humidity.get_predictions(
        LAG, 
        TEMPERATURE_WEEKS,
        RAIN_WEEKS,
        AVGRH_WEEKS,
        real,
        ws,
        province)
    
    plot_all_predictions(LAG, 
                         real,
                         train,
                         province,
                         [1,2,4,8,16],
                         predictions_to_plot)


In [3]:
all_weather = pd.read_csv("weather01-16.csv")
all_weather.stn_name = all_weather.stn_name.apply(dengue_weather.remove_space)
all_dengue = pd.read_csv("all-dengues.csv")
all_dengue = all_dengue.rename(columns = {'จังหวัด' : 'province'})

In [4]:
                       # province_dengue: province_weather
provinces_to_predict = {"Bangkok":"BangkokMetropolis", 
                        "Nonthaburi":"BangkokMetropolis",
                        "PhraNakhonSiAyutthaya":"AyuttayaAgromet", 
                        "Phuket":"Phuket", 
                        "Ratchaburi":"Ratchaburi", 
                        "ChonBuri":"ChonBuri", 
                        "NongKhai":"NongKhai",
                        "SakonNakhon":"SakonNakhonAgromet",
                        "ChiangMai":"ChiangMai",
                        "ChiangRai":"ChiangRai"}

province_data = {}
for province_for_dengue in provinces_to_predict:
    province_for_weather = provinces_to_predict.get(province_for_dengue)
    print province_for_dengue, province_for_weather
    
    province_data_to_predict = dengue_weather.get_dengue_weather_by_province(
                                                                    province_for_dengue, 
                                                                    province_for_weather, 
                                                                    all_dengue, 
                                                                    all_weather)
    province_data[province_for_dengue] = province_data_to_predict
    
# province data is dictionary of province to its train and real data, idx 0 and 1
# ex --> province_data.get('Bangkok')[0]

Bangkok BangkokMetropolis
Phuket Phuket
ChonBuri ChonBuri
PhraNakhonSiAyutthaya AyuttayaAgromet
Nonthaburi BangkokMetropolis
NongKhai NongKhai
SakonNakhon SakonNakhonAgromet
Ratchaburi Ratchaburi
ChiangMai ChiangMai
ChiangRai ChiangRai


In [5]:
for i in province_data:
    print province_data[i][0].head(1)
    break

            avgrh  dday  meantemp  rain  cases
date                                          
2003-01-05  71.25   3.5      29.1   0.0     12


In [ ]:
import multiprocessing as mp
from multiprocessing import Process, Queue

# Define an output queue
output = Queue()

ws_csv = np.array(pd.read_csv("best-with-19-lags-9-temps-15-rains-1-humidity-ws-2nd-iter-model-a.csv").T)[0]
    
def plot_by_province(province, train, real, output, LAG = 19, TEMPERATURE_WEEKS = 9, RAIN_WEEKS = 15, AVGRH_WEEKS = 1):
    
#     print province, train.head(1)
#     print
#     print province, real.head(1)
#     print 
#     print province, train.tail(1)
#     print
#     print province, real.tail(1)
#     print 
    
    LAG, TEMPERATURE_WEEKS, RAIN_WEEKS, AVGRH_WEEKS = 19, 9, 15, 1
                                    # get_alphas takes 4 args, if theres no input csv
    alphas = weather_all_alphas.get_alphas(
        LAG, 
        TEMPERATURE_WEEKS, 
        RAIN_WEEKS, 
        AVGRH_WEEKS,
        train, 
        ws_csv)
                                    # get_predictions takes 6 args
    predictions_to_validate = prediction_humidity.get_predictions(
        LAG, 
        TEMPERATURE_WEEKS, 
        RAIN_WEEKS, 
        AVGRH_WEEKS, 
        real, 
        alphas.x, 
        province)
                                    # get_validations takes 3 argsg
    validated_result = validate.get_validations(
        LAG, 
        predictions_to_validate, 
        real)
    
    output.put((
        province,
        predictions_to_validate,
        validated_result,
        alphas.x)) # add in ws as well alphas.x


In [ ]:
%%time
# Setup a list of processes that we want to run


processes = [Process(
                target=plot_by_province, 
                args=(
                    province, 
                    province_data[province][0], 
                    province_data[province][1], 
                    output)\
                ) for province in province_data]

print ws_csv

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

[  3.19682069e-01   0.00000000e+00   0.00000000e+00   4.09892183e-10
   4.62719860e-10   7.80728653e-09   2.53650003e-08   2.13348863e-08
   9.67920880e-09   2.70368108e-08   3.43607541e-08   2.12063159e-07
   3.29800618e-08   1.00133190e-01   3.63057496e-02   2.42642130e-08
   5.39429557e-08   6.70643583e-09   9.85639456e-08   4.50562838e-01
   2.11949778e+00   7.97772722e+00   2.87382041e-01   5.26851984e-02
  -8.29951381e-03  -4.02833313e-02  -2.64725012e-02  -1.26135592e-02
  -1.82857910e-02   3.63907496e-03   3.23924132e-03  -8.50589679e-03
   5.08167612e-02   1.44065615e-04   6.97311738e-05  -2.23398445e-05
   3.39118832e-04  -4.50725469e-05  -2.28152952e-09  -2.21192325e-06
   1.62489516e-06   1.19664963e-06  -2.24543089e-08   1.71177569e-06
  -2.18340401e-05  -2.22055979e-08  -1.81890893e-07   8.85878490e-09
   9.83738122e-02   1.95761544e-06]
50
LAG, TEMP, RAIN, AVGRH 19 9 15 1
50
bnds 19 50
bnds 19 50
LAG, TEMP, RAIN, AVGRH 19 9 15 1
ws_csv
50
ws_csv
50
LAG, TEMP, RAIN, AVGRH